# Validity Organized

In [1]:
import os,sys
import numpy as np
import pandas as pd
from scipy import stats

import warnings; warnings.filterwarnings('ignore');

src_dir = os.path.abspath(os.pardir)
sys.path[0] = src_dir

from analysis.CodingScoring import scoreTIPI, normTIPI, stratTIPI

In [2]:
questions = pd.read_csv('../data/External/dataHeadings.csv')
dataRaw = pd.read_csv('../data/External/dataRaw.csv', names=questions.QuestionCode)
# dataRaw.head()

In [3]:
df = dataRaw.join(normTIPI(scoreTIPI(dataRaw)))
# df.head()

In [4]:
cxOCEAS = df[['Chronotype','O','C','E','A','S']]
# cxOCEAS.to_csv('chronotypeOCEAS.csv')
# cxOCEAS.head()

In [45]:
cxOCEAS.groupby('Chronotype').mean().transpose()

Chronotype,Lark,Owl,Strong lark,Strong owl
O,-0.250813,-0.222538,-0.239012,-0.062307
C,0.451232,0.151161,0.596748,-0.081510
E,-0.198122,-0.231596,-0.232154,-0.247742
A,0.164111,0.072580,0.164584,-0.020602
S,0.188074,0.099233,0.223488,-0.042507


One way ANOVA on continuous data

In [13]:
for trait in ['O','C','E','A','S']:
    sl = cxOCEAS[trait][cxOCEAS['Chronotype']=='Strong lark'].dropna()
    l = cxOCEAS[trait][cxOCEAS['Chronotype']=='Lark'].dropna()
    o = cxOCEAS[trait][cxOCEAS['Chronotype']=='Owl'].dropna()
    so = cxOCEAS[trait][cxOCEAS['Chronotype']=='Strong owl'].dropna()
    f_val, p_val = stats.f_oneway(sl,l,o,so)
    print('F=%f, p=%f'%(f_val,p_val))

F=30.401000, p=0.000000
F=300.886551, p=0.000000
F=1.638169, p=0.178211
F=26.590246, p=0.000000
F=42.857438, p=0.000000


In [48]:
cxOCEAS['N'] = 0-cxOCEAS['S']
cxOCEAN = cxOCEAS.drop(columns=['S']).dropna()
cxOCEAN_strat = stratTIPI(cxOCEAN,3)

In [54]:
pd.crosstab(index=cxOCEAN_strat['N_strat'],columns=cxOCEAN_strat['Chronotype'])

Chronotype,Lark,Owl,Strong lark,Strong owl
N_strat,,,,
-1,1195,1538,388,869
0,2788,3756,732,2120
1,470,838,148,697


Chi-square on stratified data

In [8]:
for trait in ['O','C','E','A','N']:
    x = pd.crosstab(index=cxOCEAN_strat['%s_strat'%trait],columns=cxOCEAN_strat['Chronotype'])
    chi2, p, dof, expected = stats.chi2_contingency(x.values)
    print('Chi2=%f, p=%f, dof=%f'%(chi2,p,dof))

Chi2=72.095988, p=0.000000, dof=6.000000
Chi2=705.254558, p=0.000000, dof=6.000000
Chi2=52.354679, p=0.000000, dof=6.000000
Chi2=83.631618, p=0.000000, dof=6.000000
Chi2=139.858501, p=0.000000, dof=6.000000


Investigate Extraversion

In [12]:
chi = pd.crosstab(index=cxOCEAN_strat['E_strat'],columns=cxOCEAN_strat['Chronotype'])
chi[['Strong lark','Lark','Owl','Strong owl']]

Chronotype,Strong lark,Lark,Owl,Strong owl
E_strat,,,,
-1,341,1090,1564,1077
0,725,2767,3768,2044
1,202,596,800,565


In [20]:
anova = cxOCEAN_strat[['Chronotype','E']].groupby(['Chronotype']).mean().transpose()
anova[['Strong lark','Lark','Owl','Strong owl']]

Chronotype,Strong lark,Lark,Owl,Strong owl
E,-0.231328,-0.195902,-0.229726,-0.244788


### Favorite color

In [5]:
std_colors = ['blue','green','black','yellow','purple','red','orange','pink']

In [6]:
df_color_OCEAN = df[['FavColor','O','C','E','A','S']][df['FavColor'].str.lower().str.strip().isin(std_colors)]

In [6]:
df_color_OCEAN['FavColor'] = df_color_OCEAN['FavColor'].str.lower().str.strip()

In [7]:
df_color_OCEAN['N'] = 0-df_color_OCEAN['S']
df_color_OCEAN = df_color_OCEAN.drop(columns='S')
df_color_OCEAN = df_color_OCEAN.dropna()

In [46]:
df_color_OCEAN.groupby('FavColor').mean().transpose()

FavColor,black,blue,green,orange,pink,purple,red,yellow
O,-0.162288,-0.266999,-0.184459,-0.048483,-0.372666,-0.222334,-0.202441,-0.166028
C,0.099729,0.260581,0.235971,0.187997,0.264777,0.209272,0.233502,0.267389
E,-0.314979,-0.253572,-0.277718,-0.094479,-0.123006,-0.192427,-0.137823,-0.130981
A,-0.229243,0.075490,0.101812,0.065918,0.137048,0.190280,0.067201,0.121796
N,0.080780,-0.198233,-0.132225,-0.098176,0.265020,0.001422,-0.145637,-0.010801
O_strat,-0.097865,-0.135864,-0.099234,-0.043299,-0.176056,-0.107710,-0.117253,-0.092342
C_strat,0.021352,0.086478,0.075885,0.028866,0.089202,0.066510,0.080402,0.087838
E_strat,-0.181495,-0.139746,-0.153594,-0.059794,-0.091549,-0.108888,-0.069514,-0.069820
A_strat,-0.033808,0.153763,0.167457,0.140206,0.185446,0.211301,0.130653,0.175676
N_strat,-0.019573,-0.170153,-0.129880,-0.107216,0.086854,-0.064155,-0.148241,-0.054054


In [35]:
for trait in ['O','C','E','A','N']:
    b = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='blue'].dropna()
    g = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='green'].dropna()
    bl = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='black'].dropna()
    y = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='yellow'].dropna()
    p = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='purple'].dropna()
    r = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='red'].dropna()
    o = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='orange'].dropna()
    pk = df_color_OCEAN[trait][df_color_OCEAN['FavColor']=='pink'].dropna()
    f_val, p_val = stats.f_oneway(b,g,bl,y,p,r,o,pk)
    print('F=%f, p=%f'%(f_val,p_val))

F=6.150761, p=0.000000
F=2.534130, p=0.013251
F=5.468164, p=0.000003
F=11.231668, p=0.000000
F=20.434863, p=0.000000


Chi square

In [36]:
df_color_OCEAN_strat = stratTIPI(df_color_OCEAN,3)

In [47]:
for trait in ['O','C','E','A','N']:
    x = pd.crosstab(index=df_color_OCEAN_strat['%s_strat'%trait],
                    columns=df_color_OCEAN_strat['FavColor'])
    chi2, p, dof, expected = stats.chi2_contingency(x.values)
    print('Chi2=%f, p=%f, dof=%f'%(chi2,p,dof))

Chi2=38.505321, p=0.000435, dof=14.000000
Chi2=23.142801, p=0.057985, dof=14.000000
Chi2=42.015415, p=0.000123, dof=14.000000
Chi2=82.264428, p=0.000000, dof=14.000000
Chi2=127.733672, p=0.000000, dof=14.000000


In [59]:
pd.crosstab(index=df_color_OCEAN_strat['N_strat'],columns=df_color_OCEAN_strat['FavColor'])

FavColor,black,blue,green,orange,pink,purple,red,yellow
N_strat,,,,,,,,
-1,129,1318,714,121,56,383,333,99
0,315,2790,1669,295,277,1042,705,270
1,118,529,358,69,93,274,156,75
